<a href="https://colab.research.google.com/github/KimEunChae9506/2024-smartfarm-AI-strongberry/blob/main/%EC%82%AC%EC%A0%84%ED%85%8C%EC%8A%A4%ED%8A%B8_%EC%B5%9C%EC%A2%85%EC%A0%9C%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# 생육 데이터 불러오기
growth_file = '사전테스트-생육데이터.xlsx'

# Daraframe형식으로 엑셀 파일 읽기
growth_data  = pd.read_excel(growth_file)

# 환경 데이터 불러오기
farm_files = ['environmentsB.csv', 'environmentsC.csv', 'environmentsD.csv', 'environmentsE.csv']
environment_data = pd.DataFrame()

for file_name in farm_files:
    temp = pd.read_csv(file_name, sep = ',', encoding = 'utf-8')
    environment_data = pd.concat([environment_data, temp])

# 데이터 클리닝
growth_data['조사일자'] = pd.to_datetime(growth_data['조사일자'], format='%Y%m%d')
environment_data['datetime'] = pd.to_datetime(environment_data['datetime'])

# 데이터 병합. 생육데이터에 없는 날짜는 환경데이터와 가까운 날짜로 채워넣음
environment_data.rename(columns={'farm': '시설아이디'}, inplace=True)
merged_data = pd.merge_asof(growth_data.sort_values('조사일자'), environment_data.sort_values('datetime'),
                            by='시설아이디', left_on='조사일자', right_on='datetime', direction='nearest')

# 데이터 전처리
merged_data = merged_data.dropna()

# 특성과 타겟 변수 선택
X = merged_data[['supplyEC', 'supplyPH', 'innerCO2', 'innerHum', 'innerTemp', 'innerSolar']]
y = merged_data['조사항목값']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 회귀 모델 생성 및 훈련
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

rf_pred = rf_model.predict(X_test)

import joblib

#모델 저장
joblib.dump(rf_model, 'saved_model.pkl')

results_df = pd.DataFrame(
    {   '시설아이디': merged_data['시설아이디'].values,
        '생육주사': merged_data['생육주사'].values,
        '조사일자': merged_data['조사일자'].values,
        '표본번호': merged_data['표본번호'].values,
        '조사항목': merged_data['조사항목'].values,
        '단위': merged_data['단위'].values,
        '조사항목값': merged_data['조사항목값'].values
    }
)

# 결과를 CSV 파일로 저장
results_df.to_csv('predicted_growth.csv', index=False)


# MASE 계산 (numpy 사용)
def mase(actual, predicted, seasonality=1):
    n = len(actual)
    d = np.mean(np.abs(np.diff(actual, n=seasonality)))
    errors = np.abs(actual - predicted)
    return np.mean(errors) / d

mase_value = mase(y_test.values, rf_pred)

print("MASE:", mase_value)

MASE: 0.7757592286819476
